In [2]:
%pip install torch transformers scikit-learn numpy


  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached safetensors-0.5.3-cp38-abi3-macosx_11_0_arm64.whl.metadata (3.8 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 MB 26.3 MB/s eta 0:00:0000:0100:01
Using cached transformers-4.51.3-py3-none-any.whl (10.4 MB)
Using cached safetensors-0.5.3-cp38-abi3-macosx_11_0_arm64.whl (418 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 34.9 MB/s eta 0:00:00
Using cached tokenizers-0.21.1-cp39-abi3-macosx_11_0_arm64.whl (2.7 MB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [12]:
import os
import pandas as pd

dir=os.getcwd()

#Lets set the path to the data
data_path = os.path.join(dir, 'Data')
#Lets set the path to the model
calssification_model_path = os.path.join(dir, 'classification_models')
#Lets check if this exists and if not then create it'
if not os.path.exists(data_path):
    os.makedirs(data_path)
if not os.path.exists(calssification_model_path):
    os.makedirs(calssification_model_path)
#Lets set the path for the validation images
validation_images_path = os.path.join(data_path, 'Validation Images')
#Lets set the path for the Frames from Video
frames_from_video_path = os.path.join(data_path, 'Frames from Video')

#Lets print the paths
print("Data Path: ", data_path)
print("Validation Images Path: ", validation_images_path)
print("Frames from Video Path: ", frames_from_video_path)


threat_analysis_path = os.path.join(dir, 'threat_analysis.xlsx')
# lets set the responses path
response_path = os.path.join(data_path, 'Responses')
if not os.path.exists(response_path):
    print("Responses not found at: ", response_path)
#Lets set the images path
images_path = os.path.join(data_path, 'Images')
if not os.path.exists(images_path):
    print("Images not found at: ", images_path)
#Set the api URL
URL = "http://127.0.0.1:8000/analyze_image_agent"


Data Path:  /Users/chetankothari/VS Projects/Threat_Analyzer_Autogen_Multiagent/Data
Validation Images Path:  /Users/chetankothari/VS Projects/Threat_Analyzer_Autogen_Multiagent/Data/Validation Images
Frames from Video Path:  /Users/chetankothari/VS Projects/Threat_Analyzer_Autogen_Multiagent/Data/Frames from Video


In [13]:

#Lets load the threat analysis data to dataframe
threat_analysis_df = pd.read_excel(threat_analysis_path, sheet_name=None)
#Lets get the sheet names
sheet_names = threat_analysis_df.keys()
#Lets print the sheet names
print("Sheet Names: ", sheet_names)
#Lets get the first sheet
sheet_names_list = list(sheet_names)  # Convert dict_keys to a list
threat_analysis_df = threat_analysis_df[sheet_names_list[0]]
#Lets print the first 5 rows
print(threat_analysis_df.head())



Sheet Names:  dict_keys(['Sheet1'])
               Image Name                                        Description  \
0   coba_train (3267).jpg  Description:\n**Scenario Overview**\n\nThe ima...   
1  coba_train_bg (47).jpg  Description:\n**Scenario Assessment**\n\n*   *...   
2          Non_Threat.jpg  Description:\n**Military Scenario Assessment**...   
3   coba_train (2677).jpg  Description:\n**Military Scenario Assessment**...   
4   coba_train (4473).jpg  Description:\n**Scene Type:** Battlefield\n\n*...   

                                             Objects  \
0  Objects Detected:\nThe image presents a collec...   
1  Objects Detected:\nThe image depicts a small, ...   
2  Objects Detected:\nThe image shows a large gre...   
3  Objects Detected:\nThe image depicts a cannon ...   
4  Objects Detected:\nThe image depicts a person ...   

                                           Reasoning       Threat Class  
0  Reasoning:\nOkay, so I'm trying to figure out ...  Threat Level: 0\n 

In [14]:
df_temp = threat_analysis_df.copy()
# Drop the file extension from the image names
df_temp['Image Name'] = df_temp['Image Name'].str.replace('.jpg', '', regex=False)
# Now lets drop the unwanted text from column threat class
df_temp['Threat Class'] = df_temp['Threat Class'].str.extract(r'(\d+)').astype(int)
# Lets drop the unwanted columns    
df_temp.drop(columns=['Objects', 'Reasoning'], inplace=True)
print(df_temp.head())
print(df_temp.__len__)

           Image Name                                        Description  \
0   coba_train (3267)  Description:\n**Scenario Overview**\n\nThe ima...   
1  coba_train_bg (47)  Description:\n**Scenario Assessment**\n\n*   *...   
2          Non_Threat  Description:\n**Military Scenario Assessment**...   
3   coba_train (2677)  Description:\n**Military Scenario Assessment**...   
4   coba_train (4473)  Description:\n**Scene Type:** Battlefield\n\n*...   

   Threat Class  
0             0  
1             0  
2             0  
3             0  
4             3  
<bound method DataFrame.__len__ of             Image Name                                        Description  \
0    coba_train (3267)  Description:\n**Scenario Overview**\n\nThe ima...   
1   coba_train_bg (47)  Description:\n**Scenario Assessment**\n\n*   *...   
2           Non_Threat  Description:\n**Military Scenario Assessment**...   
3    coba_train (2677)  Description:\n**Military Scenario Assessment**...   
4    coba_train

In [15]:
#Lets copy the dataframe back the threat_analysis_df
threat_analysis_df = df_temp.copy()
print(threat_analysis_df.head())
print(threat_analysis_df.__len__)
print(threat_analysis_df.columns)

           Image Name                                        Description  \
0   coba_train (3267)  Description:\n**Scenario Overview**\n\nThe ima...   
1  coba_train_bg (47)  Description:\n**Scenario Assessment**\n\n*   *...   
2          Non_Threat  Description:\n**Military Scenario Assessment**...   
3   coba_train (2677)  Description:\n**Military Scenario Assessment**...   
4   coba_train (4473)  Description:\n**Scene Type:** Battlefield\n\n*...   

   Threat Class  
0             0  
1             0  
2             0  
3             0  
4             3  
<bound method DataFrame.__len__ of             Image Name                                        Description  \
0    coba_train (3267)  Description:\n**Scenario Overview**\n\nThe ima...   
1   coba_train_bg (47)  Description:\n**Scenario Assessment**\n\n*   *...   
2           Non_Threat  Description:\n**Military Scenario Assessment**...   
3    coba_train (2677)  Description:\n**Military Scenario Assessment**...   
4    coba_train

In [17]:
#Now lets prepare another data from from responses of multiagent analysis
#Lets load the threat analysis data to dataframe the responses are set of json files which we would read to create a dataframe with each record being samed with the image name
#Lets read the response files
image_files = os.listdir(images_path)
#Lets set the response files path
response_files = os.listdir(response_path)
print("Response Files: ", response_files)
#Lets create a empty dataframe
response_df = pd.DataFrame()
#Lets loop through the response files
import os
import json
import pandas as pd

# Initialize a list to store the data
data = []
# Loop through each file in the folder
for image_file in image_files:
    response_file = os.path.join(response_path, os.path.splitext(image_file)[0] + '.json')
    if os.path.exists(response_file):
        with open(response_file, 'r', encoding='utf-8') as f:
            response_data = json.load(f)
        #Extract the Vision Analysis data
        vision_analysis = response_data.get('vision_analysis', {})
        # Extract and handle the threat level
        threat_level = response_data.get('threat_level', 'N/A')
        if isinstance(threat_level, str):
            # Handle cases where threat_level is a string
            numeric_threat_level = threat_level.split("–")[0].strip()  # Extract the numeric part
        elif isinstance(threat_level, dict):
            # Handle cases where threat_level is a dictionary
            numeric_threat_level = threat_level.get('threatLevel', 'N/A')
        else:
            # Handle unexpected cases
            numeric_threat_level = 'N/A'

        # Append the image name and numeric threat level to the data list
        data.append({"Image Name": image_file,"Vision_Analysis":vision_analysis,"Threat Level": numeric_threat_level})
# Create a DataFrame from the collected data
multiagent_threat_df = pd.DataFrame(data)
# Drop the file extension from the image names 
multiagent_threat_df['Image Name'] = multiagent_threat_df['Image Name'].str.replace('.jpg', '', regex=False)

# Display the DataFrame
print(multiagent_threat_df)

Response Files:  ['coba_train (5454).json', 'coba_train (1402).json', 'coba_train (3184).json', 'coba_train (3510).json', 'coba_train (5344).json', 'coba_train (4795).json', 'coba_train (4867).json', 'coba_train (1638).json', 'Non_Threat.json', 'coba_train (5413).json', 'coba_train (972).json', 'coba_train (2182).json', 'coba_train (2984).json', 'coba_train (4307).json', 'coba_train (1015).json', 'coba_train (4958).json', 'coba_train (5452).json', 'coba_train_bg (34).json', 'coba_train (3267).json', 'coba_train (194).json', 'coba_train (2575).json', 'coba_train (1603).json', 'coba_train (3735).json', 'coba_train_bg (12).json', 'coba_train (418).json', 'coba_train (5692).json', 'coba_train (4473).json', 'coba_train (3102).json', 'coba_train (1559).json', 'coba_train (2187).json', 'coba_train (1479).json', 'coba_train (5468).json', 'coba_train (5298).json', 'coba_train (4548).json', 'coba_train (3594).json', 'coba_train (3695).json', 'coba_train_bg (33).json', 'coba_train (4434).json', '

In [18]:
print(multiagent_threat_df.columns)

Index(['Image Name', 'Vision_Analysis', 'Threat Level'], dtype='object')


In [ ]:
#Prepare the data for training
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel

# Load BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

# Function to convert text to BERT embeddings
def extract_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()  # Mean pooling of token embeddings

# Convert all descriptions to BERT embeddings
import json

# Ensure all entries in the Vision_Analysis column are strings
multiagent_threat_df["Vision_Analysis"] = multiagent_threat_df["Vision_Analysis"].apply(
    lambda x: json.dumps(x) if isinstance(x, dict) else str(x)
)

# Convert all descriptions to BERT embeddings
X = torch.tensor(
    [np.array(extract_bert_embeddings(desc), dtype=np.float32) for desc in multiagent_threat_df["Vision_Analysis"]]
)
# Ensure all values in the "Threat Level" column are integers
multiagent_threat_df["Threat Level"] = pd.to_numeric(multiagent_threat_df["Threat Level"], errors="coerce").fillna(0).astype(int)

# Convert threat levels to numerical values
y = torch.tensor(multiagent_threat_df["Threat Level"].values)  # Labels



/var/folders/61/39zk8n_x5l32mb7rjkls_4n00000gn/T/ipykernel_10931/2989730700.py:27: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:257.)
  X = torch.tensor(


In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [21]:
import torch.nn as nn
import torch.optim as optim

# Define MLP model
class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_dim, output_dim)  # Final layer

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

# Model hyperparameters
input_dim = 768  # BERT embedding size
hidden_dim = 2096  # Hidden layer size
output_dim = 6  # Number of threat classes (0-5)

# Initialize model, loss function, optimizer
model = MLPClassifier(input_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [22]:
num_epochs = 100
batch_size = 8

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for i in range(0, len(X_train), batch_size):
        batch_X = X_train[i:i+batch_size]
        batch_y = y_train[i:i+batch_size]

        optimizer.zero_grad()
        outputs = model(batch_X.float())
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss:.4f}")


Epoch 1/100, Loss: 13.1788
Epoch 2/100, Loss: 10.0495
Epoch 3/100, Loss: 9.6183
Epoch 4/100, Loss: 9.5567
Epoch 5/100, Loss: 9.3507
Epoch 6/100, Loss: 9.1926
Epoch 7/100, Loss: 9.0519
Epoch 8/100, Loss: 8.8713
Epoch 9/100, Loss: 8.6890
Epoch 10/100, Loss: 8.4819
Epoch 11/100, Loss: 8.2464
Epoch 12/100, Loss: 7.9954
Epoch 13/100, Loss: 7.7197
Epoch 14/100, Loss: 7.4275
Epoch 15/100, Loss: 7.1225
Epoch 16/100, Loss: 6.8069
Epoch 17/100, Loss: 6.4881
Epoch 18/100, Loss: 6.1774
Epoch 19/100, Loss: 5.8664
Epoch 20/100, Loss: 5.5703
Epoch 21/100, Loss: 5.2991
Epoch 22/100, Loss: 5.0403
Epoch 23/100, Loss: 4.7928
Epoch 24/100, Loss: 4.5650
Epoch 25/100, Loss: 4.3528
Epoch 26/100, Loss: 4.1521
Epoch 27/100, Loss: 3.9612
Epoch 28/100, Loss: 3.7724
Epoch 29/100, Loss: 3.5913
Epoch 30/100, Loss: 3.4158
Epoch 31/100, Loss: 3.2604
Epoch 32/100, Loss: 3.1017
Epoch 33/100, Loss: 2.9494
Epoch 34/100, Loss: 2.8067
Epoch 35/100, Loss: 2.6608
Epoch 36/100, Loss: 2.5190
Epoch 37/100, Loss: 2.3825
Epoch 38

In [23]:
from sklearn.metrics import accuracy_score

model.eval()
with torch.no_grad():
    y_pred = model(X_test.float()).argmax(dim=1)

accuracy = accuracy_score(y_test.numpy(), y_pred.numpy())
print(f"Test Accuracy: {accuracy:.2f}")


Test Accuracy: 0.65


In [24]:
def extract_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)  # Make sure this is the BERT model, not the MLP model
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()  # Mean pooling of token embeddings


In [25]:
# Load BERT separately to avoid overwriting
bert_model = AutoModel.from_pretrained("bert-base-uncased")

def extract_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = bert_model(**inputs)  # Use bert_model instead of model
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()


In [26]:
new_text = "An armed individual is moving near the perimeter."
# Extract BERT embeddings for the new text
new_embedding = extract_bert_embeddings(new_text)  # Already returns a numpy array
new_embedding = torch.tensor(new_embedding).float().unsqueeze(0)  # Convert to tensor and add batch dimension

# Pass the embeddings to the MLPClassifier
model.eval()
with torch.no_grad():
    predicted_threat = model(new_embedding).argmax(dim=1).item()

print(f"Predicted Threat Level: {predicted_threat}")


Predicted Threat Level: 4


In [9]:
import os
import json
import requests
import pandas as pd
validation_response_path = os.path.join(data_path, 'validation_responses')
if not os.path.exists(validation_response_path):
    os.makedirs(validation_response_path)
exts = ('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff')
for fname in os.listdir(frames_from_video_path):
    if not fname.lower().endswith(exts):
        continue
    
    in_path  = os.path.join(frames_from_video_path, fname)
    out_name = os.path.splitext(fname)[0] + '.json'
    out_path = os.path.join(validation_response_path, out_name)
    
    with open(in_path, 'rb') as img_file:
        # service expects the image under "file"
        files = {'file': (fname, img_file, 'application/octet-stream')}
        try:
            resp = requests.post(URL, files=files, data={'request_type': 'vision_analysis'})
            resp.raise_for_status()
        except requests.RequestException as e:
            print(f"[ERROR] {fname}: {e}")
            continue
    
    # write JSON (or raw text on parse failure)
    try:
        data = resp.json()
        with open(out_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
    except ValueError:
        with open(out_path, 'w', encoding='utf-8') as f:
            f.write(resp.text)
    
    print(f"[OK]   {fname} → {out_name}")


[OK]   videoframe_0.png → videoframe_0.json
[OK]   videoframe_52051.png → videoframe_52051.json
[OK]   Test_new.png → Test_new.json
[OK]   videoframe_6565.png → videoframe_6565.json


In [ ]:
# Now lets predict the threat level for the validation images based on the responses
validation_responses = os.listdir(validation_response_path)

for response in validation_responses:
    response_file = os.path.join(validation_response_path, response)
    with open(response_file, 'r', encoding='utf-8') as f:
        response_data = json.load(f)
    # Extract the Vision Analysis data
    vision_analysis = response_data.get('vision_analysis', {})
    # Extract BERT embeddings for the new text
    vision_embedding = extract_bert_embeddings(json.dumps(vision_analysis))  # Convert dictionary to string
    vision_embedding = torch.tensor(vision_embedding).float().unsqueeze(0)  # Convert to tensor and add batch dimension

    # Pass the embeddings to the MLPClassifier
    model.eval()
    with torch.no_grad():
        predicted_threat = model(vision_embedding).argmax(dim=1).item()

    print(f"Predicted Threat Level: {response}:{predicted_threat}")


Predicted Threat Level: videoframe_52051.json:1
Predicted Threat Level: Test_new.json:3
Predicted Threat Level: videoframe_0.json:0
Predicted Threat Level: videoframe_6565.json:0


In [33]:
# 1. Install dependencies (locally):
#    pip install torch transformers

from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import json

# Assume multiagent_threat_df is already loaded in the notebook
# with columns ["Vision_Analysis", "Threat Level"]

# Initialize BERT
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")
model.eval()

# Function to get embeddings
def get_bert_embedding(text: str) -> np.ndarray:
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    # mean-pool token embeddings
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy().squeeze()

# Prepare data
# Convert Vision_Analysis dicts to strings
multiagent_threat_df["Vision_Analysis"] = multiagent_threat_df["Vision_Analysis"].apply(
    lambda x: json.dumps(x) if isinstance(x, dict) else str(x)
)

# Extract embeddings
embs = np.vstack([get_bert_embedding(txt) for txt in multiagent_threat_df["Vision_Analysis"]])

# Labels
y = pd.to_numeric(multiagent_threat_df["Threat Level"], errors="coerce").fillna(0).astype(int).values

# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    embs, y, test_size=0.2, random_state=42, stratify=y
)

# Define classifiers
models = {
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "RandomForest": RandomForestClassifier(n_estimators=200),
    "SVM": SVC(kernel="linear", probability=True),
}

# Evaluate
results = []
for name, clf in models.items():
    print(f"Training {name}...")
    clf.fit(X_train, y_train)
    cv = cross_val_score(clf, X_train, y_train, cv=5, scoring="f1_weighted")
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    macro_f1 = classification_report(y_test, y_pred, output_dict=True)["macro avg"]["f1-score"]
    results.append({
        "Model": name,
        "CV F1 Mean": cv.mean(),
        "CV F1 Std": cv.std(),
        "Test Accuracy": acc,
        "Test Macro F1": macro_f1
    })

results_df = pd.DataFrame(results)
results_df.style.set_caption("BERT Embedding Classifier Comparison")


Training LogisticRegression...


/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels wi

Training RandomForest...


/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels wi

Training SVM...


,Model,CV F1 Mean,CV F1 Std,Test Accuracy,Test Macro F1
0,LogisticRegression,0.547736,0.129272,0.705882,0.496212
1,RandomForest,0.488791,0.105984,0.588235,0.258929
2,SVM,0.519079,0.037573,0.529412,0.429110


Error encoding image ...: [Errno 63] File name too long: '...'
Error getting description for image ...: name 'base64_image' is not defined
